In [2]:
# Behlül Gülmez
# 150140113

#Shoppers' refuse rate too low, i didnt see any in my tests.
#I used and take help from week5 examples from ninova, architecture is very smilar with that example.
#Even started cabbages are rotting.
#When a shopper comes, he/she buys a cabbage which is closest to rotting time so rotting of cabbages stays minimum.

In [16]:
import random

In [17]:
def fel_key(event):
    return event.event_time

In [18]:
class Event:
    
    def __init__(self,event_time):
        self.event_time = event_time
    
    def process(self,state,fel,clock):
        raise NotImplementedError
        
    def __str__(self):
        return (str(self.__class__.__name__) + ":" + str(round(self.event_time, 2)))
        
class OrderCabbageEvent(Event):
    
    def __init__(self,event_time):
        super().__init__(event_time)
    
    def process(self,state,fel,clock):#add delivercabbage event with deliver time into future event list
        fel.append(DeliverCabbageEvent(event_time=clock+round(random.uniform(1, 15), 2)))          

class DeliverCabbageEvent(Event):
    
    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):
        #if we have already 40 cabbages, we wont take this but it never happened in my tests.
        if state['cabbages_in_stock']<40:
            state['cabbages_in_stock']+=1
            fel.append(StartRottingEvent(event_time=clock))

class StartRottingEvent(Event):

    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):#add finishrotting event with rotting time into future event list
        fel.append(FinishRottingEvent(event_time=clock+round(random.uniform(7, 12), 2)))

class FinishRottingEvent(Event):

    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):
        state['cabbages_in_stock']-=1#I dont need to check <0 because if there is no cabbage it cannot rot. I calculated rotting times each for cabbages seperatly.
        fel.append(OrderCabbageEvent(event_time=clock))
    
class ShopperArriveEvent(Event):

    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):#shoppers arrives 1 by 1 continuesly
        fel.append(ShopperArriveEvent(event_time=clock+round(random.uniform(0, 3), 2)))
        if state['cabbages_in_stock']>0:
            fel.append(HappyShopperEvent(event_time=clock))           
        else:
            fel.append(UnhappyShopperEvent(event_time=clock))

class HappyShopperEvent(Event):
    
    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):
        state['cabbages_in_stock']-=1
        fel.append(OrderCabbageEvent(event_time=clock))
        
        #if stock is too low order more
        if state['cabbages_in_stock'] < 20:
            fel.append(OrderCabbageEvent(event_time=clock))
        elif state['cabbages_in_stock'] < 10:
            fel.append(OrderCabbageEvent(event_time=clock))
            fel.append(OrderCabbageEvent(event_time=clock))
        elif state['cabbages_in_stock'] < 5:
            fel.append(OrderCabbageEvent(event_time=clock))
            fel.append(OrderCabbageEvent(event_time=clock))
            fel.append(OrderCabbageEvent(event_time=clock))
        
        #ignoring first cabbage rot if we sell a cabbage
        for i in range(0,len(fel)):
            if str(fel[i])[0:6] == 'Finish':
                del fel[i]
                break      
        
class UnhappyShopperEvent(Event):
    
    def __init__(self,event_time):
        super().__init__(event_time)
        
    def process(self,state,fel,clock):
        pass


In [19]:
fel = []#future event list array
clock = 0#timer
state = {'cabbages_in_stock':30}#starting stock
for i in range(0,30):#rot times for starter cabbages
    fel.append(StartRottingEvent(event_time=clock))
first_event = ShopperArriveEvent(event_time=0)#first shopper arrives at 0.0
fel.append(first_event)

In [20]:
i = 1
while len(fel)!=0 and clock<100:#simulate until clock 100
    fel.sort(key=fel_key)#sort array by clock
    next_event = fel[0]
    del fel[0]#pop used event
    print(str(i) + ": state:" + str(state) + "    next event: " +str(next_event))
    clock = next_event.event_time
    next_event.process(state,fel,clock)
    i+=1

1: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
2: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
3: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
4: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
5: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
6: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
7: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
8: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
9: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
10: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
11: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
12: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
13: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
14: state:{'cabbages_in_stock': 30}    next event: StartRottingEvent:0
15: state:{'cab